<a href="https://colab.research.google.com/github/lyw02/EtymologyLM/blob/main/nb/Qwen3_(4B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Goal: To convert `Qwen3-4B-Base` into a reasoning model via GRPO by using OpenR1's Math dataset.

We first pre fine-tune the model to make GRPO skip trying to match formatting - this speeds GRPO up.

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-16 02:28:57 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-16 02:28:57 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.4: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 49.43%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill t

tokenizer_config.json:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.5.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### GRPO chat template
Since we're using a base model, we should set a chat template. You can make your own chat template as well!
1. DeepSeek uses `<think>` and `</think>`, but this is **not** necessary - you can customize it however you like!
2. A `system_prompt` is recommended to at least guide the model's responses.

In [4]:
# reasoning_start = "<start_working_out>" # Acts as <think>
# reasoning_end   = "<end_working_out>"   # Acts as </think>
# solution_start  = "<SOLUTION>"
# solution_end    = "</SOLUTION>"
structure_start = "<structure>"
structure_end   = "</structure>"
content_start   = "<content>"
content_end     = "</content>"

system_prompt = \
f"""You are a helpful linguist, especially expertise in English etymology.
The user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.
Always output ONLY the structured information. No extra words.
Always remember, in side the output, etymology structure should be placed between {structure_start} and {structure_end}, and detailed content of each node should be placed between {content_start} and {content_end}.
If the input is not relevant to etymology, you should output CANNOT_RECOGNISE."""
system_prompt

'You are a helpful linguist, especially expertise in English etymology.\nThe user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.\nAlways output ONLY the structured information. No extra words.\nAlways remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.\nIf the input is not relevant to etymology, you should output CANNOT_RECOGNISE.'

We create a simple chat template below. Notice `add_generation_prompt` includes prepending `<start_working_out>` to guide the model to start its reasoning process.

In [5]:
chat_template = \
    "{{ '{system_prompt}' + eos_token }}"\
    "{% for message in messages[1:] %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")
tokenizer.chat_template = chat_template

Let's see how our chat template behaves on an example:

In [6]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : """
Summary
A borrowing from Latin.
Etymon: Latin formātiōn-em.
< Latin formātiōn-em, noun of action < formāre to form v.1: see ‑ation suffix.
    """},
    {"role" : "assistant", "content" : f"""
{structure_start}
English - Latin 1 - Latin 2
{structure_end}
{content_start}
@English: {{formation}}
@Latin 1: {{formātiōn-em}} [noun of action]
@Latin 2: {{formāre}} [to form v.1]
{content_end}
    """},
], tokenize = False, add_generation_prompt = True)

'You are a helpful linguist, especially expertise in English etymology.\nThe user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.\nAlways output ONLY the structured information. No extra words.\nAlways remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.\nIf the input is not relevant to etymology, you should output CANNOT_RECOGNISE.<|endoftext|>\n<structure>\nEnglish - Latin 1 - Latin 2\n</structure>\n<content>\n@English: {formation}\n@Latin 1: {formātiōn-em} [noun of action]\n@Latin 2: {formāre} [to form v.1]\n</content>\n    <|endoftext|>'

### Pre fine-tuning for formatting
We now use a subset of NVIDIA's [Open Math Reasoning dataset](https://huggingface.co/datasets/nvidia/OpenMathReasoning) which was filtered to only include high quality DeepSeek R1 traces.

We'll only filter ~59 or so examples to first "prime" / pre fine-tune the model to understand our custom GRPO formatting.

In [7]:
from datasets import load_dataset
import pandas as pd
import numpy as np


dataset = load_dataset("lyw02/ety-100-core", split = "train")
df = dataset.to_pandas()
df_exp = df.explode("messages")
msg_df = pd.json_normalize(df_exp["messages"])
msg_df["conv_id"] = df_exp.index
agg = (
    msg_df
    .groupby(["conv_id", "role"])["content"]
    .apply(list)
    .unstack(fill_value=[])
)
agg["user"] = agg["user"].str[0]
agg["assistant"] = agg["assistant"].str[0]

dataset = agg.reset_index(drop=True)[["user", "assistant"]]

dataset = dataset.head(50)

dataset

annotated_100_train.jsonl:   0%|          | 0.00/225k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

role,user,assistant
0,Summary\nOf multiple origins. Partly a borrowi...,<structure>\nEnglish - Anglo-Norman & Old Fren...
1,Summary\nOf multiple origins. Partly a borrowi...,<structure>\nEnglish - Middle French\nEnglish ...
2,Summary\nA word inherited from Germanic.\nCogn...,<structure>\nEnglish - Old Frisian - Germanic\...
3,Summary\nOf multiple origins. Partly a borrowi...,<structure>\nEnglish - Anglo-Norman & Middle F...
4,Summary\nA borrowing from French.\nEtymon: Fre...,<structure>\nEnglish - Anglo-Norman - Latin 1 ...
5,Summary\nA borrowing from French.\nEtymon: Fre...,<structure>\nEnglish - Middle English - Old Fr...
6,Summary\nOf multiple origins. Partly a borrowi...,<structure>\nEnglish - Anglo-Norman & Old Fren...
7,Summary\nA borrowing from French.\nEtymon: Fre...,<structure>\nEnglish - Old French - Latin 1 - ...
8,Summary\nA borrowing from French.\nEtymon: Fre...,<structure>\nEnglish - Old French 1 - Old Fren...
9,Summary\nProbably a borrowing from French.\nNo...,<structure>\nEnglish - Latin ?- French\n</str...


We have to format the dataset to follow our GRPO style formatting:

In [8]:
def format_dataset(x):
    # expected_answer = x["expected_answer"]
    # problem = x["problem"]

    user = x["user"]
    assistant = x["assistant"]

    # Remove generated <think> and </think>
    # thoughts = x["generated_solution"]
    # thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    # Strip newlines on left and right
    # thoughts = thoughts.strip()
    # Add our custom formatting
    # final_prompt = \
    #     reasoning_start + thoughts + reasoning_end + \
    #     solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : user},
        {"role" : "assistant", "content" : assistant},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

Check to see if it worked:

In [9]:
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

'You are a helpful linguist, especially expertise in English etymology.\nThe user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.\nAlways output ONLY the structured information. No extra words.\nAlways remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.\nIf the input is not relevant to etymology, you should output CANNOT_RECOGNISE.<|endoftext|>Summary\nOf multiple origins. Partly a borrowing from French. Partly a borrowing from Latin.\nEtymons: French espirit, spirit; Latin spīritus.\n< (i) Anglo-Norman esperite, espirite, espirith, (rare) spirit, Anglo-Norman and Old French, Middle French esperit, espirit (Anglo-Norman and Middle French, French esprit) animating or vital principle, wind, breath, air, action of breathing, divine inspiration, consciousness, em

Let's truncate the pre fine-tuning dataset to `max_seq_length/2` since we don't want too long reasoning traces.

Note this might take 2 minutes!

In [10]:
dataset["N"] = dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

# dataset = dataset.loc[dataset["N"] <= max_seq_length/2].copy()
dataset.shape

(50, 4)

We then tokenize the messages and convert it to a Hugging Face compatible dataset format:

In [11]:
from datasets import Dataset

dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['user', 'assistant', 'Messages', 'N', 'text'],
    num_rows: 50
})

Let's now pre fine-tune the model so it follows our custom GRPO formatting!

In [12]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 10, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/50 [00:00<?, ? examples/s]

In [13]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 10 | Total steps = 500
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 66,060,288/4,088,528,384 (1.62% trained)


Step,Training Loss
10,1.992900
20,1.067400
30,0.914200
40,0.950200
50,0.821800
60,0.707200
70,0.494800
80,0.736000
90,0.592000
100,0.555300


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=500, training_loss=0.2699145928621292, metrics={'train_runtime': 280.581, 'train_samples_per_second': 1.782, 'train_steps_per_second': 1.782, 'total_flos': 6943357150003200.0, 'train_loss': 0.2699145928621292})

Let's check if the model has learnt to follow the custom format:

In [14]:
text = tokenizer.apply_chat_template(
    dataset[0]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

dataset[0]["Messages"][:2]

You are a helpful linguist, especially expertise in English etymology.
The user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.
Always output ONLY the structured information. No extra words.
Always remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.
If the input is not relevant to etymology, you should output CANNOT_RECOGNISE.<|endoftext|>Summary
Of multiple origins. Partly a borrowing from French. Partly a borrowing from Latin.
Etymons: French espirit, spirit; Latin spīritus.
< (i) Anglo-Norman esperite, espirite, espirith, (rare) spirit, Anglo-Norman and Old French, Middle French esperit, espirit (Anglo-Norman and Middle French, French esprit) animating or vital principle, wind, breath, air, action of breathing, divine inspiration, consciousness, emotion, t

[{'content': 'You are a helpful linguist, especially expertise in English etymology.\nThe user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.\nAlways output ONLY the structured information. No extra words.\nAlways remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.\nIf the input is not relevant to etymology, you should output CANNOT_RECOGNISE.',
  'role': 'system'},
 {'content': 'Summary\nOf multiple origins. Partly a borrowing from French. Partly a borrowing from Latin.\nEtymons: French espirit, spirit; Latin spīritus.\n< (i) Anglo-Norman esperite, espirite, espirith, (rare) spirit, Anglo-Norman and Old French, Middle French esperit, espirit (Anglo-Norman and Middle French, French esprit) animating or vital principle, wind, breath, air, action of breathing,

Run inference

In [15]:
import re

def extract_and_format_content(input_str: str) -> str:
    """
    Extract contents in <structure> and <content> tags, and return a new formatted string
    """
    input_str = input_str.split("<|endoftext|>")[1]
    structure_match = re.search(r"<structure>(.*?)</structure>", input_str, re.DOTALL)
    s_content = structure_match.group(1) if structure_match else ""

    content_match = re.search(r"<content>(.*?)</content>", input_str, re.DOTALL)
    c_content = content_match.group(1) if content_match else ""

    new_string = f"<structure>\n{s_content}\n</structure>\n<content>\n{c_content}\n</content>"

    return new_string


from unsloth.chat_templates import get_chat_template
import json
from time import time

with open('test_set_small.jsonl', 'r', encoding='utf-8') as f_in, open('inference.jsonl', 'w', encoding='utf-8') as f_out:

    eta = []
    for line in f_in:
        data = json.loads(line)
        messages = [
            {
              "role": "system",
              "content": system_prompt
            },
            {
              "role": "user",
              "content": data['messages'][1]["content"]
            }
        ]
        # print(messages)


        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt = True, # Must add for generation
        )

        t_start = time()
        outputs = model.generate(
            **tokenizer([text], return_tensors = "pt").to("cuda"),
            max_new_tokens = 4096, # Increase for longer outputs!
            # Recommended Gemma-3 settings!
            temperature = 1.0, top_p = 0.95, top_k = 64,
        )
        t_end = time()
        eta.append(t_end - t_start)

        reply = tokenizer.batch_decode(outputs)
        # print(reply)

        formatted_reply = extract_and_format_content(reply[0])
        # print(formatted_reply)

        output = [{}, {}]
        output.append({"role": "assistant", "content": formatted_reply})
        json.dump({"messages": output}, f_out)
        f_out.write('\n')
        print("Proceeded line")

print(f"ETA: {sum(eta)/len(eta)}")

Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
ETA: 9.143427443504333


In [16]:
with open('annotated_50_grpo.jsonl', 'r', encoding='utf-8') as f_in, open('inference_for_grpo_train_set.jsonl', 'w', encoding='utf-8') as f_out:

    eta = []
    for line in f_in:
        data = json.loads(line)
        messages = [
            {
              "role": "system",
              "content": system_prompt
            },
            {
              "role": "user",
              "content": data['messages'][1]["content"]
            }
        ]
        # print(messages)


        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt = True, # Must add for generation
        )

        t_start = time()
        outputs = model.generate(
            **tokenizer([text], return_tensors = "pt").to("cuda"),
            max_new_tokens = 4096, # Increase for longer outputs!
            # Recommended Gemma-3 settings!
            temperature = 1.0, top_p = 0.95, top_k = 64,
        )
        t_end = time()
        eta.append(t_end - t_start)

        reply = tokenizer.batch_decode(outputs)
        # print(reply)

        formatted_reply = extract_and_format_content(reply[0])
        # print(formatted_reply)

        output = [{}, {}]
        output.append({"role": "assistant", "content": formatted_reply})
        json.dump({"messages": output}, f_out)
        f_out.write('\n')
        print("Proceeded line")

print(f"ETA: {sum(eta)/len(eta)}")

Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
Proceeded line
ETA: 11.110818099975585


Yes it did follow the formatting! Great! Let's remove some items before the GRPO step

In [17]:
del dataset
torch.cuda.empty_cache()
import gc
gc.collect()

299

### Data Prep
<a name="Data"></a>

We're using Hugging Face's [Open R1 Math dataset](https://huggingface.co/datasets/open-r1/DAPO-Math-17k-Processed). You can also utilize OpenAI's famous [GSM8K dataset](https://huggingface.co/datasets/openai/gsm8k)

In [18]:
dataset = load_dataset("lyw02/ety-100-core", split = "train")

dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": x["messages"][1]["content"]},
    ],
    "answer": x["messages"][2]["content"],
}, remove_columns=dataset.column_names)

dataset = dataset.select(range(50, 100))
dataset

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'answer'],
    num_rows: 50
})

Let's look at the first row:

In [19]:
dataset[0]["prompt"]

[{'content': 'You are a helpful linguist, especially expertise in English etymology.\nThe user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.\nAlways output ONLY the structured information. No extra words.\nAlways remember, in side the output, etymology structure should be placed between <structure> and </structure>, and detailed content of each node should be placed between <content> and </content>.\nIf the input is not relevant to etymology, you should output CANNOT_RECOGNISE.',
  'role': 'system'},
 {'content': "Summary\nA borrowing from French.\nEtymon: French descouvrir.\n< Anglo-Norman descoverir, descovrir, descuverir, discoverir, discovrir, discuverir, discouvrir, descoverer, discoverer, descoverre, Anglo-Norman and Old French, Middle French descovrir, Anglo-Norman and Middle French descouvrir (compare Anglo-Norman decoverir, decovrir, decuvrir; Middle French, French découvrir) to expo

We create a regex format to match the reasoning sections and answers:

In [20]:
import re

# Add optional EOS token matching
structure_end_regex = r"</structure>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

content_end_regex = r"</content>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_structure = re.compile(
    rf"{structure_start}(.+?){structure_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

match_content = re.compile(
    rf"{content_start}(.+?){content_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

We verify it works:

In [21]:
str_to_match_ = """
<structure>
English - Latin 1 - Latin 2
</structure>
<content>
@English: {{formation}}
@Latin 1: {{formātiōn-em}} [noun of action]
@Latin 2: {{formāre}} [to form v.1]
</content>
"""

match_structure.findall(str_to_match_)

['\nEnglish - Latin 1 - Latin 2\n']

In [22]:
match_content.findall(str_to_match_)

['\n@English: {{formation}}\n@Latin 1: {{formātiōn-em}} [noun of action]\n@Latin 2: {{formāre}} [to form v.1]\n']

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [23]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_structure.search(response) is not None: score += 3.0
        if match_content.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [24]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!

        # No need to reward <start_working_out> since we always prepend it!
        # score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(structure_end)   == 1 else -1.0
        score += 0.5 if response.count(content_start)  == 1 else -1.0
        score += 0.5 if response.count(content_end)    == 1 else -1.0
        scores.append(score)
    return scores

Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [25]:
!pip install networkx

import networkx as nx
from networkx.algorithms import isomorphism
import re


def parse_structure(input_string):
    """
    解析图结构，支持普通连接、特殊连接和子图。
    输入示例：
    A - B - (C + D)
    C - E
    返回节点集合和边列表。
    """
    def parse_line(line):
        # 分割字符串，捕获分隔符
        pattern = r'(\s-\s|\s-\S+-\s)'
        parts = re.split(pattern, line.strip())
        edges = []
        i = 0
        while i < len(parts) - 2:  # 确保有源节点、分隔符、目标节点
            source = parts[i].strip()
            target = parts[i + 2].strip()
            if target.startswith('(') and target.endswith(')'):
                # 处理子图
                subgraph_str = target[1:-1].strip()  # 移除括号
                subgraph_nodes = [node.strip() for node in subgraph_str.split(' + ')]
                for node in subgraph_nodes:
                    edges.append((node, source))  # 反转方向
            else:
                # 普通连接
                edges.append((target, source))  # 反转方向
            i += 2  # 跳到下一个源节点
        return edges

    # 分割输入字符串为行
    lines = [line.strip() for line in input_string.splitlines() if line.strip()]
    all_edges = []
    for line in lines:
        edges = parse_line(line)
        all_edges.extend(edges)

    # 提取节点
    nodes = set()
    for edge in all_edges:
        nodes.add(edge[0])  # 源节点
        nodes.add(edge[1])  # 目标节点

    return nodes, all_edges


def build_graph(nodes, edges):
    """构建有向图"""
    G = nx.DiGraph()
    for node in nodes:
        G.add_node(node, label=node)
    for edge in edges:
        if len(edge) == 2:
            G.add_edge(edge[0], edge[1])
        elif len(edge) == 3:
            G.add_edge(edge[0], edge[1], **edge[2])
    return G


def compute_f1_score(gt_str, inf_str):
  nodes1, edges1 = parse_structure(gt_str)
  g1 = build_graph(nodes1, edges1)
  nodes2, edges2 = parse_structure(inf_str)
  g2 = build_graph(nodes2, edges2)

  # 提取边集
  edges_g1 = set(g1.edges())
  edges_g2 = set(g2.edges())

  # 计算共同边（交集）
  common_edges = edges_g1.intersection(edges_g2)

  # 计算精度
  if len(edges_g2) > 0:
      precision = len(common_edges) / len(edges_g2)
  else:
      precision = 0.0  # 避免除以零

  # 计算召回率
  if len(edges_g1) > 0:
      recall = len(common_edges) / len(edges_g1)
  else:
      recall = 0.0  # 避免除以零

  # 计算F1 score
  if precision + recall > 0:
      f1_score = 2 * (precision * recall) / (precision + recall)
  else:
      f1_score = 0.0

  return f1_score

In [43]:
def load_true_answer():
  contents = []
  with open("inference_for_grpo_train_set.jsonl", 'r', encoding='utf-8') as f:
      for line in f:
          data = json.loads(line)
          content = data['messages'][2]['content']
          contents.append(content)
  return contents


def check_answer(prompts, completions, answer, **kwargs):
    # true_answers = load_true_answer()
    # print(f"prompts: {prompts}")
    # print(f"completions: {completions}")
    # print(f"answer: {answer}")

    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_structure.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(-2.0)
            continue

        true_answer = match_structure.findall(true_answer)[0]

        # print(f"Question: {question}")
        # print(f"True Answer: {true_answer}")
        # print(f"Response: {guess}")

        score += compute_f1_score(true_answer, guess) * 10

        scores.append(score)
    return scores

In [33]:
true_answers = load_true_answer()
# for answer in true_answers:
#   print(match_structure.findall(answer)[0])
compute_f1_score(true_answers[1], "English 1 - (English 2 + English 3)")

1.0

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [46]:
# max_prompt_length = maximum_length + 1 # + 1 just in case!
# max_completion_length = max_seq_length - max_prompt_length
max_prompt_length = 4096
max_completion_length = 4096

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    num_train_epochs = 1, # Set to 1 for a full training run
    # max_steps = 1000,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        # match_format_exactly,
        # match_format_approximately,
        check_answer,
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 66,060,288/4,088,528,384 (1.62% trained)


<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [30]:
system_prompt = \
f"""You are a helpful linguist, especially expertise in English etymology.
The user will give you a piece of text that describes etymology of a word, you must output its structured etymology information in the given pattern.
Always output ONLY the structured information. No extra words.
Always remember, in side the output, etymology structure should be placed between {structure_start} and {structure_end}, and detailed content of each node should be placed between {content_start} and {content_end}.
If the input is not relevant to etymology, you should output CANNOT_RECOGNISE."""

text = system_prompt + """
Summary\nA borrowing from French.\nEtymon: French cunfort.\n< Old French cunfort, confort (11th cent. in Littré) = Italian conforto, Old Spanish conforto, a noun apparently of Romanic age, from stem of confortāre, Old French conforter to comfort v. It took the place of Old English frofor, with which it is used indifferently in enumerating the nine urouren or ‘comforts’ against temptations, in Ancren Riwle p. 226 seq.\n"},{"role":"assistant","content":"<structure>\nEnglish - Old French 1 - Old French 2 - Romanic\nItalian - Romanic\nOld Spanish - Romanic\nEnglish -replaced- Old English\n</structure>\n<content>\n@English: {{comfort}}\n@Old French 1: {{cunfort}} {{confort}} ({$time(11th cent.) 11th cent. in Littré})\n@Old French 2: {{conforter}} [to comfort v.]\n@Romanic: {{confortāre}}\n@Italian: {{conforto}}\n@Old Spanish: {{conforto}}\n@Old English: {{frofor}}\n</content>\n
"""

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

'<structure>\n@English -Old French 1 -cunfort, confort\n@Old French 1 -conforter -to comfort v. (ca. 11th cent.)\n@Romanic -confortāre\n@Italian -conforto\n@Old Spanish -conforto\n@Old English -frofor\n</structure>\n<content>\n\n@Comfort of (Old English) frofor and Latin comfertāre, comfertum (cf. comfort) in the sense of ‘comfort, relief,’ ‘consolation,’ ‘benefit,’ from PIE *kom-pwrod-ti- ‘to take, carry,’ suffixed form of PIE *cemp- “to put” (root of Latin emittō, immittō “give up,” immitto “send forth; discharge, give up”). In a technical sense “the power to make changes in the behavior of an organism” (1955), from French confort psychologique “behavior in a state of psychological ‘comfort’” (as in “comfort zone”). “To console,” also from Old French in Late Latin via Italian. Conformism 1 comes from “fit, match,” so called from the similarity (conforn) to a standard. Conformism 2 (“conform to standard, regulations, etc.”) is from “give in, agree” (OE cunnaran, cf. Old Frisian, Dutch

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"Okay, so I need to find the square root of 101. Hmm, I remember that square roots are about finding a number that, when multiplied by itself, gives the original number. But 101 isn't a perfect square, right? Let me think. The squares around 101 would be 100 (which is 10 squared) and 121 (which is 11 squared). So the square root of 101 is somewhere between 10 and 11. \n\nBut the problem is asking for the square root, which might be an irrational number. So maybe it's a decimal? Let me try to estimate. I need to find the number that's closer to 10.5. Let me test 10.05. If I square that, it's (10 + 0.05)*(10 + 0.05) = 100 + 0.05*10 + 0.05*10 + 0.05*0.05 = 100 + 0.5 + 0.5 + 0.0025 = 101.0025. That's very close to 101. So maybe the square root is 10.05 plus or minus a little bit. \n\nBut wait, maybe there's a better way. Since 10.05 squared is 101.0025, which is slightly above 101, the actual square root should be slightly less than 10.05. Let me try 10.04. Squaring 10.04: (10 + 0.04)*(10 

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
